In [1]:
#!pip uninstall numpy -y

In [2]:
#!pip uninstall transformers torch torchvision numpy -y

In [13]:
# clone https://github.com/christinach/solr9-semantic.git to use sample data and python script
!git clone https://github.com/christinach/solr9-semantic.git

fatal: destination path 'solr9-semantic' already exists and is not an empty directory.


In [17]:
cd solr9-semantic/

/content/solr9-semantic


In [60]:
# pull recent changes that were added in solr9-semantic repo
!git pull origin main

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 7 (delta 6), reused 7 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 1.61 KiB | 8.00 KiB/s, done.
From https://github.com/christinach/solr9-semantic
 * branch            main       -> FETCH_HEAD
   8700f8a..4fa390d  main       -> origin/main
Updating 8700f8a..4fa390d
Fast-forward
 data/marc_values.json |  6 +++---
 data/marc_values.txt  | 20 ++++++++++----------
 sample_of_records.xml |  3 ++-
 small_sample.xml      |  3 ++-
 4 files changed, 17 insertions(+), 15 deletions(-)


In [ ]:
# run this in the solr9-semantic repo
# !python3 solr9-semantic/text_create.py

In [3]:
!pip install transformers torch torchvision numpy --upgrade

In [41]:
!pip install pandas
import pandas as pd

In [61]:
# ls data/marc_values.json

In [56]:
# use this to troubleshoot
# import pandas as pd

# # Read JSON file and convert values to array
# df = pd.read_json('data/marc_values.json', orient='index')
# values_array = df[0].values.tolist()

# # Print first few values to verify
# print("First 3 values:")
# for value in values_array[:3]:
#     print(f"{value[:100]}...")  # Print first 100 characters of each value

# print(f"\nTotal number of values: {len(values_array)}")



First 3 values:
British writing from empire to Brexit : writing, identity and nation / Robert Spencer, Howard J. Boo...
Contesting cosmopolitan moments in the long eighteenth century / Enit Karafili Steiner. Edinburgh : ...
Radical writers and the media revolution in the late Enlightenment / Christoph Streb. Liverpool : Li...

Total number of values: 40


In [57]:
# Show versions:
!pip freeze | grep tensorflow
!pip freeze | grep torch
!pip freeze | grep numpy
# Show Python version
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

tensorflow==2.18.0
tensorflow-datasets==4.9.8
tensorflow-hub==0.16.1
tensorflow-io-gcs-filesystem==0.37.1
tensorflow-metadata==1.17.1
tensorflow-probability==0.25.0
tensorflow-text==2.18.1
tensorflow_decision_forests==1.11.0
torch @ https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchao==0.10.0
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision @ https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-linux_x86_64.whl
numpy==2.0.2
/usr/bin/python3
3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
sys.version_info(major=3, minor=11, micro=12, releaselevel='final', serial=0)


In [63]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Check PyTorch version
print(torch.__version__)
print(torch.cuda.is_available)

# torch.set_default_device(device='cuda:0')
# torch.get_default_device()

# print(torch.accelerator.current_accelerator().type)

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

df = pd.read_json('data/marc_values.json', orient='index')
# values_array = df[0].values.tolist()
texts = df[0].values.tolist()

def get_sentence_embedding(text):
    # Tokenize and prepare input tensors
    inputs = tokenizer(text, return_tensors='pt')
    # Disable gradient calculation
    with torch.no_grad():
        # Get model outputs
        outputs = model(**inputs)
    # Extract the last hidden states (embeddings)
    last_hidden_states = outputs.last_hidden_state
    # Average token embeddings
    sentence_embedding = torch.mean(last_hidden_states, dim=1).numpy()
    return sentence_embedding

# Generate embeddings for texts
embeddings = [get_sentence_embedding(text) for text in texts]

# Query text
query_text = "American literature History"
query_embedding = get_sentence_embedding(query_text)

# Compute cosine similarities
# Lokk for another similarity index that I could use.
similarities = cosine_similarity(query_embedding, np.vstack(embeddings))

# Print query text
print (f"Query text: {query_text}")

# Print similarities
for i, text in enumerate(texts):
    print(f"Similarity with '{text}': {similarities[0][i]}")


2.6.0+cu124
<function is_available at 0x7f56e5929b20>
Query text: American literature History
Similarity with 'British writing from empire to Brexit : writing, identity and nation / Robert Spencer, Howard J. Booth, and Anastasia Valassopoulos. Abingdon, Oxon ; New York, NY : Routledge, Taylor & Francis Group, 2025. ©2025 National characteristics, British, in literature. European Union Great Britain. Great Britain In literature. Literature and society England History 19th century. Literature and society England History 20th century. English literature 19th century History and criticism. English literature 20th century History and criticism. Britanniques dans la littérature. (CaQQLa)000278323 Littérature et société (CaQQLa)201-0044931 Angleterre (CaQQLa)201-0476579 Histoire (CaQQLa)201-0378908 19e siècle. (CaQQLa)201-0378908 Littérature et société (CaQQLa)201-0044931 Angleterre (CaQQLa)201-0476579 Histoire (CaQQLa)201-0378892 20e siècle. (CaQQLa)201-0378892 Littérature anglaise